In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import pysentiment2 as ps
import nltk
import pysentiment2 as ps
from nltk.stem.porter import *
import os

In [2]:
#Load the Returns file
df = pd.read_csv('../DataRetrieval/url_with_returns.csv')[['reportDate','filingDate', 'cik', 'ExcessReturnsEqualWeightedSnP', 'ExcessReturnsValueWeightedSnP']]
# df = pd.read_csv('../DataRetrieval/url_with_returns.csv')[['reportDate','filingDate', 'cik', 'ExcessReturnsEqualWeightedSnP', 'ExcessReturnsValueWeightedSnP']]
df['cik'] = df['cik'].astype(np.int).astype(str).str.pad(10, fillchar='0')
# Filtering out returns on or after 2016-01-01
df = df[df.reportDate >= '2016-01-01']
print(df.shape)

(11383, 5)


C:\Users\JuliJaramillo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\JuliJaramillo\AppData\Local\Temp\ipykernel_24144\203754680.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df['cik'] = df['cik'].astype(np.int).astype(str).str.pad(10, fillchar='0')


In [3]:
#Define the dictionaries

LMDictionary = pd.read_excel('LM_Neg.xlsx', sheet_name = 'ND_FinTerms_Negative_v2', header = None).values.reshape(-1)
LMDictionary = set( [w.lower().strip() for w in LMDictionary ] )

HarvardDictionary = list(open('Harvard_Neg_Inf.txt', 'r'))
HarvardDictionary = set ( [w.strip().lower() for w in HarvardDictionary ] )

#Define the stopwords dictionary
stop_words = list(set(stopwords.words('english')))

C:\Users\JuliJaramillo\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
#listFileNames = os.listdir('../Forms/')
listFileNames = os.listdir('../DataRetrieval/Forms/')

# Calculates the total number of documents under consideration
N = 0
# Calculate the number of times a 
globalDict = {}

In [5]:
for row, fileName in enumerate(listFileNames):
    
    cik = fileName.split('_')[0]
    filingDate = fileName.split('_')[1].split('.')[0]
    #path = '../Forms/{}_{}.txt'.format(cik, filingDate)

    path = '../DataRetrieval/Forms/{}_{}.txt'.format(cik, filingDate)
    
    f = open(path, "r")
    text = f.read()
    
    tokensOwn = nltk.word_tokenize(text)
    
    # Filtering out the documents where the numbers of words are less than 2000
    if len(tokensOwn) >= 2000:       
        N+=1
        filteredText = [w for w in tokensOwn if not w in stop_words]
        
        # Updated the global_dict to include count of words observed in the current document
        #Number of documents that contains the token "word"
        #Df_i
        for word in list(set(filteredText)):
            if not word in globalDict:
                globalDict[word] = 1
            else:
                globalDict[word] += 1
        
    f.close()
    
    if (row % 1000 == 0) & (row != 0): 
        print(row)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [6]:
def wordCountDictionary(documentTokens):
    '''
    Counting occurrences of a given word in a document. EVERY WORD, not just the negative words. (TF_ij)
        documentTokens: (list) tokens of the document
    '''
    
    documentWordCountDict = {}
    for word in list(documentTokens):
        if not word in documentWordCountDict:
            documentWordCountDict[word] = 1
        else:
            documentWordCountDict[word] += 1
            
    return documentWordCountDict     


In [7]:
def countNegativeWords(documentTokens, setNegativeWords):
    countNegativeWords = 0
    for word in documentTokens:
        if word in setNegativeWords:
            countNegativeWords+=1
    return countNegativeWords

In [8]:
def weightedNegativeWords(documentTokens, setNegativeWords, wordWeights):
    weightedNegativeWords = 0.0
    weightedWords = 0.0
    
    for word in documentTokens:
        weightedWords += wordWeights[word]
        if word in setNegativeWords:
            weightedNegativeWords += wordWeights[word]
            
    return weightedNegativeWords/weightedWords
    

In [9]:
for row, fileName in enumerate(listFileNames):
    
    cik = fileName.split('_')[0]
    filingDate = fileName.split('_')[1].split('.')[0]
    #path = '../Forms/{}_{}.txt'.format(cik, filingDate)

    path = '../DataRetrieval/Forms/{}_{}.txt'.format(cik, filingDate)
    
    f = open(path, "r")
    text = f.read()
    
    tokensOwn = nltk.word_tokenize(text)
    
    # Filtering out the documents where the numbers of words are less than 2000
    if len(tokensOwn) >= 2000:       
        filteredText = [w for w in tokensOwn if not w in stop_words]
        
        numeratorLM = countNegativeWords(filteredText, LMDictionary)
        propLM = numeratorLM/len(filteredText)
        
        numeratorHar = countNegativeWords(filteredText, HarvardDictionary)
        propHar = numeratorHar/len(filteredText)
        
        df.loc[(df['cik']== cik) & (df['filingDate'] == filingDate), 'propLM'] = propLM 
        df.loc[(df['cik']== cik) & (df['filingDate'] == filingDate), 'propHar'] = propHar
        
        
        wCountDictionary = wordCountDictionary(filteredText)
        a = np.mean(list(wCountDictionary.values()))
        
        wordWeights = {}
        
        for word, word_count in wCountDictionary.items():
            wordWeights[word] = (1 + np.log(wCountDictionary[word]))*(np.log(N/globalDict[word]))/(1 + np.log(a))
        
        
        weightedLM = weightedNegativeWords(filteredText, LMDictionary, wordWeights)
        weightedHar = weightedNegativeWords(filteredText, HarvardDictionary, wordWeights)
        
        df.loc[(df['cik']== cik) & (df['filingDate'] == filingDate), 'weightedLM'] = weightedLM 
        df.loc[(df['cik']== cik) & (df['filingDate'] == filingDate), 'weightedHar'] = weightedHar 
                
    f.close()
    
    if (row % 1000 == 0) & (row != 0): 
        print(row)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [10]:
final = df[df['propLM'].notna()]

In [11]:
final.to_csv('proportions.csv')